In [ ]:
import pandas as pd
import numpy as np
from pandas import *
from numpy import *
import re
from numpy import linalg as la  

In [ ]:
#计算时间
def count_time(s1,s2):
    t1=int(s1.split(':')[0])*60*60+int(s1.split(':')[1])*60+int(s1.split(':')[2])
    t2=int(s2.split(':')[0])*60*60+int(s2.split(':')[1])*60+int(s2.split(':')[2])
    if t2<t1:
        t2+=24*60*60
    return t2-t1

#求余弦
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

#先求整体相似矩阵
def similarMat(dataMat,simMeas=cosSim):
    U,Sigma,VT = la.svd(dataMat)
    Sig2=Sigma**2
    for i in range(Sig2.shape[0]): #找到达到90能量的奇异值数目
        if sum(Sig2[:i]) > sum(Sig2)*0.9:
            s=i
            break
    n = shape(dataMat)[1]
    m = shape(dataMat)[0]
    M=np.zeros((n,n))
    Sig = mat(eye(s)*Sigma[:s]) #arrange Sig into a diagonal matrix  
    xformedItems = dataMat.T * U[:,:s] * Sig.I  #create transformed items  
    for i in range(n):
        for j in range(i+1,n):  
            similarity = simMeas(xformedItems[i,:].T,xformedItems[j,:].T)
            M[i,j]=similarity
            M[j,i]=similarity
            print('the %d and %d similarity is: %f'%(i, j, similarity))
    return M
#奇异值分解
def svdEst(dataMat, user, simMat, item):
    simTotal = 0.0; ratSimTotal = 0.0
    n = shape(dataMat)[1]
    for j in range(n): 
        userRating = dataMat[user,j]  
        if userRating == 0 or j==item: continue  
        similarity = simMat[item,j]
        simTotal += similarity  
        ratSimTotal += similarity * userRating  
    if simTotal == 0: return 0  
    else: return ratSimTotal/simTotal
#simMat=similarMat(mat(A)) #求出相似矩阵
#按照用户推荐
def recommend(dataMat,user,simMat,N=3,simMeas=cosSim,estMethod=svdEst):
    #寻找未评级的物品  
    unratedItems=nonzero(dataMat[user,:].A==0)[1]  
    if len(unratedItems)==0: return 'you rated everything'  
    itemScores=[]  
    for item in unratedItems: 
        estimatedScore=estMethod(dataMat,user,simMat,item) #对每一个未评分物品，调用standEst()来产生该物品的预测得分  
        itemScores.append((item,estimatedScore)) #该物品的编号和估计得分值放入一个元素列表itemScores中  
    #对itemScores进行从大到小排序，返回前N个未评分物品  
    return sorted(itemScores,key=lambda jj:jj[1],reverse=True)[:N]  

In [ ]:
df=pd.read_csv('点播信息.csv',encoding='gbk')

#整理节目名称
for i in range(len(df)):
    print(i)
    st=df.loc[i,'影片名称']
    st=re.sub('：.*','',st)
    st=re.sub('\(\S+\)','',st)
    st=re.sub('【\S+】','',st)
    st=re.sub('\S+月\S+日','',st)
    df.loc[i,'name']=st.strip()

#整理用户号
a=list(set(df.loc[:,'用户号'].tolist()))
a.sort()
dic={}
for i in range(len(a)):
    dic[a[i]]=i

#整理矩阵
z=pd.DataFrame(0,index=[d for d in range(len(dic))],columns=sorted(list(set(df.loc[:,'name'].tolist()))))
for i in range(len(df)):
    z.loc[dic[df.loc[i,'用户号']],df.loc[i,'name']]+=(count_time(df.loc[i,'观看开始时间'],df.loc[i,'观看结束时间'])//60)
    #z.loc[dic[df.loc[i,'用户号']],df.loc[i,'name']]=1
A=z.as_matrix()

In [ ]:
#求出整体相关矩阵
simMat=similarMat(mat(A))

In [ ]:
#按照用户推荐
recommend(mat(A),2,simMat)